<img src="../idp.jpg">

<a id='1'></a>
# <p style="background-color:#2F5597; text-font:white; font-family:newtimeroman; font-size:120%; text-align:center"><span style="color:white">📚 AULA 2 - Sistema de Recomendação de Restaurantes</span></p>

<img src="zomato.jpg">


## Objetivo do notebook
Criar um sistema de recomendação baseado em uma lista de restaurantes, tipos de cozinha e avaliações.

Funcionamento do modelo:
- Escrever um nome de restaurante
- O sistema de recomendação analisará as avaliações de outros restaurantes
- O sistema nos recomendará outros restaurantes com avaliações semelhantes e os classificará a partir dos mais bem avaliados.

### O que é
A Zomato é um serviço de busca de restaurantes para quem quer sair para jantar, buscar comida ou pedir em casa na Índia, Brasil, Portugal, Turquia, Indonésia, Nova Zelândia, Itália, Filipinas, África do Sul, Sri Lanka, Catar, Emirados Árabes Unidos, Reino Unido, Estados Unidos, Austrália e Canadá. O site estava posicionado no ranking Alexa como 99 na Índia e 595 no mundo em Outubro de 2015.

### Dados de interesse
Zomato é principalmente um aplicativo de entrega de comida, tendo parceria com 350 mil restaurantes e cafés em 526 cidades indianas. Também permite que os clientes reservem mesas para jantar, escrevam críticas sobre comida e carreguem fotos. A app Zomato tem 41,5 milhões de clientes que usam seu serviço todos os meses, e as encomendas na sua plataforma aumentaram para 403,1 milhões no ano 2019-2020.

### Dados que vamos trabalhar
O guia de restaurantes Zomato permite ao usuário buscar informações relacionadas a restaurantes, bares, cafés, pubs e casa noturnas. As informações fornecidas geralmente incluem:
- nome do estabelecimento
- telefones de contato
- endereço
- horário de funcionamento
- cardápio
- fotografias
- avaliações 
- mapas de localização
- avaliações e notas em uma escala de 1 a 5
- informações do tipo de cozinha (italiana, espanhola, mineira :) )
- custo aproximado de uma refeição para 2
- se aceita cartão de crédito
- se tem delivery
- etc.

----

## Conteúdo do notebook:

#### Parte 1: Análise Exploratória dos Dados (EDA):
1) Análise das variáveis

#### Parte 2: Engenharia e Limpeza de Variáveis
1) Adicionando novas variáveis

2) Removendo variáveis duplicadas

3) Convertendo o formato das variáveis para a etapa de modelagem, escala, etc.

#### Parte 3: Sistema de recomendação
1) Preparando os dados

2) Matriz TF-IDF

3) Testando o sistema

In [ ]:
# Importando as principais bibliotecas de manipulação e visualização de dados
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
# Lendo os 2 datasets
nomes_restaurantes = pd.read_csv('restaurant_names.csv') # restaurantes e dados cadastrais
reviews_restaurantes = pd.read_csv('restaurant_reviews.csv') # reviews dos restaurantes, comentários, etc.

<a id='1'></a>
# <p style="background-color:#2F5597; text-font:white; font-family:newtimeroman; font-size:120%; text-align:center"><span style="color:white">Parte 1: Análise Exploratória dos Dados</span></p>


In [ ]:
# Visualizando a estrutura dos nossos dados
nomes_restaurantes.head()

In [ ]:
# Visualizando a estrutura dos nossos dados
reviews_restaurantes.head()

In [ ]:
# vamos agora fazer nosso famoso EDA automatizado :)
from dataprep.eda import plot, plot_correlation, create_report, plot_missing

# Utilizando o método plot para construção do relatório EDA de forma automática dos nomes dos restaurantes
plot(nomes_restaurantes)

In [ ]:
create_report(nomes_restaurantes)

In [ ]:
# Vamos agora fazer o EDA para os reviews
plot(reviews_restaurantes)

In [ ]:
create_report(reviews_restaurantes)

## Combinando os 2 datasets

Iremos fazer um merge dos 2 datasets para facilitar nossa manipulação de dados e permitir que façamos análises mais completas.

In [ ]:
# Renomeando a coluna do nome do restaurante para ter o mesmo valor do outro conjunto de dados
reviews_restaurantes = reviews_restaurantes.rename(columns={'Restaurant': 'Name'})

# Unindo os 2 datasets:
df = pd.merge(reviews_restaurantes, nomes_restaurantes, how='left', on='Name')

# Eliminando as colunas que não vamos usar
df.drop(['Reviewer', 'Time', 'Pictures', 'Links', 'Collections'], axis=1, inplace=True)
df.head()

In [ ]:
df.info()

<a id='2'></a>
# <p style="background-color:#2F5597; text-font:white; font-family:newtimeroman; font-size:120%; text-align:center"><span style="color:white">Parte 2: Engenharia e Limpeza de Dados</span></p>


## Preparando as colunas de Cost e Rating

In [ ]:
# Alterando os tipos de dados das colunas de Cost e Rating
df['Cost'] = df['Cost'].str.replace(',', '').astype(int)
df['Rating'] = df['Rating'].str.replace('Like', '1').astype(float)
df.info()

## Manipulando os valores ausentes

In [ ]:
print('Número de registros:', len(df))
print('\nNúmero nulos para cada coluna:\n')
print(df.isnull().sum())

A coluna Rating é muito importante para o sistema de recomendação. Portanto, não vamos descartar esses valores de 38 NaN.
Vamos preencher esses valores NaN com o valor médio de classificação de cada restaurante.

In [ ]:
# Examine missing Rating values:
df['Name'][df['Rating'].isnull() == True].value_counts()

Portanto, existem apenas dois restaurantes com um total de 38 valores de classificação NaN.
Vamos ver o valor médio da classificação de cada restaurante.

In [ ]:
print('Média de avaliações para American Wild Wings: ', df['Rating'][df['Name'] == 'American Wild Wings'].mean())
print('Média de avaliações para Arena Eleven: ', df['Rating'][df['Name'] == 'Arena Eleven'].mean())
print('Média geral das avaliações: ', df['Rating'].mean())

Podemos ver que o valor médio para o valor de classificação ausente deve ser 4 (3,9 e 4,1 para cada restaurante).
Vamos preencher esses restaurantes com esses valores.

In [ ]:
df['Rating'].fillna(4, inplace=True)

# Mudando os NaN para '-'
df['Review'] = df['Review'].fillna('-')
df.isnull().sum()

## Separando os metadados (Reviews and Followers)

Separamos os números de Reviews e Followers em colunas diferentes para usá-lo mais tarde.

In [ ]:
# Preenchendo os valores nulos:
df['Metadata'].fillna('0 Review , 0 Follower', inplace=True)

# Padronizando as strings
df['Metadata'] = df['Metadata'].str.replace('Reviews', 'Review')
df['Metadata'] = df['Metadata'].str.replace('Followers', 'Follower')

df['Metadata'][df['Metadata'].str.endswith('w')] = df['Metadata'][df['Metadata'].str.endswith('w')] + ' , - Follower'

# Dividindo em duas colunas
df[['Reviews', 'Followers']] = df['Metadata'].str.split(' , ', expand=True)

# Removendo algumas strings dos reviews
df['Reviews'] = df['Reviews'].str.replace('Review', '')
df['Reviews'] = df['Reviews'].str.replace('Posts', '')
df['Reviews'] = df['Reviews'].str.replace('Post', '')

df['Followers'] = df['Followers'].str.replace('Follower', '')
df['Followers'] = df['Followers'].str.replace('-', '0')

# Mudando str para int
df[['Reviews', 'Followers']] = df[['Reviews', 'Followers']].astype(int)

# Removendo a coluna inicial
df.drop(['Metadata'], axis=1, inplace=True)

# Ordenando os restaurantes por custo
df = df.sort_values(['Name', 'Cost'], ascending=False).reset_index()
df.drop('index', axis=1, inplace=True)

In [ ]:
# Visualizando uma amostra nosso dataset
df.sample(5)

## Criando novas variáveis (Média de Ratings, Reviews e Followers)

As colunas Rating, Review e Followers representam as entradas de clientes individuais.
Vamos encontrar as médias desses valores e atribuí-los aos restaurantes.

In [ ]:
restaurants = list(df['Name'].unique())
df['Mean Rating'] = 0
df['Mean Reviews'] = 0
df['Mean Followers'] = 0

for i in range(len(restaurants)):
    df['Mean Rating'][df['Name'] == restaurants[i]] = df['Rating'][df['Name'] == restaurants[i]].mean()
    df['Mean Reviews'][df['Name'] == restaurants[i]] = df['Reviews'][df['Name'] == restaurants[i]].mean()
    df['Mean Followers'][df['Name'] == restaurants[i]] = df['Followers'][df['Name'] == restaurants[i]].mean()

In [ ]:
df.sample(3)

## Feature Scaling das 3 variáveis de média

Iremos utilizar a escala de 1-5

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range = (1,5))

df[['Mean Rating', 'Mean Reviews', 'Mean Followers']] = scaler.fit_transform(df[['Mean Rating', 'Mean Reviews', 'Mean Followers']]).round(2)

df.sample(3)

## Processamento do texto e limpeza dos dados

Usaremos a variável 'Review' e 'Cuisines' para criar um sistema de recomendação.
Portanto, precisamos preparar e limpar o texto nessas colunas.

In [ ]:
import re
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# 5 exemplos dessas colunas antes do processamento de texto:
df[['Review', 'Cuisines']].sample(5)

In [ ]:
import nltk
nltk.download('stopwords')

# Símbolos para ser substuidos pelos espaços
replace_space = re.compile('[/(){}\[\]\|@,;]')

# Símbolos a serem removidos
remove_symbols = re.compile('[^0-9a-z #+_]')

# Definição de stopwords
stopwords = set(stopwords.words('english'))

def text_preprocessing(text):
    # Converter para minúsculas
    text = text.lower()
    
    # Substituir os símbolos com espaços
    text = replace_space.sub(' ', text)
    
    # Remover os símbolos
    text = remove_symbols.sub('', text)
    
    # Remover stopwords
    text = ' '.join(word for word in text.split() if word not in stopwords)
    
    return text

In [ ]:
df['Review'] = df['Review'].apply(text_preprocessing)
df['Cuisines'] = df['Cuisines'].apply(text_preprocessing)

In [ ]:
# Colunas após o processamento
df[['Review','Cuisines']].sample(5)

## Analisando restaurantes e sua popularidade

In [ ]:
# Verificando os dados
restaurant_names = list(df['Name'].unique())
restaurant_names

In [ ]:
df_rating = df.drop_duplicates(subset='Name')
df_rating = df_rating.sort_values(by='Mean Rating', ascending=False).head(10)

plt.figure(figsize=(7,5))
sns.barplot(data=df_rating, x='Mean Rating', y='Name', palette='RdBu')
plt.title('Top 10 restaurantes com melhores notas');

In [ ]:
df_reviews = df.drop_duplicates(subset='Name')
df_reviews = df_reviews.sort_values(by='Mean Reviews', ascending=False).head(10)

plt.figure(figsize=(7,5))
sns.barplot(data=df_reviews, x='Mean Reviews', y='Name', palette='RdBu')
plt.title('Top 10 Restaurantes mais avaliados');

In [ ]:
df_followers = df.drop_duplicates(subset='Name')
df_followers = df_followers.sort_values(by='Mean Followers', ascending=False).head(10)

plt.figure(figsize=(7,5))
sns.barplot(data=df_followers, x='Mean Followers', y='Name', palette='RdBu')
plt.title('TOP 10 restaurantes com mais seguidores');

## Distribuição de Frequência das palavras:

In [ ]:
def get_top_words(column, top_nu_of_words, nu_of_word):
    
    # CountVectorizer é uma ótima ferramenta fornecida pela biblioteca scikit-learn em Python. 
    # Ele é usado para transformar um determinado texto em um vetor com base na frequência (contagem) 
    # de cada palavra que ocorre em todo o texto
    vec = CountVectorizer(ngram_range= nu_of_word, stop_words='english')
    
    bag_of_words = vec.fit_transform(column)
    
    sum_words = bag_of_words.sum(axis=0)
    
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    
    return words_freq[:top_nu_of_words]

In [ ]:
# Top 20 dupla de palavras por tipo de cozinha
list1 = get_top_words(df['Cuisines'], 20, (2,2))

df_words1 = pd.DataFrame(list1, columns=['Word', 'Count'])

plt.figure(figsize=(7,6))
sns.barplot(data=df_words1, x='Count', y='Word')
plt.title('Frequência de pares de palavras para os tipos de cozinhas');

In [ ]:
# Top 20 dupla de palavras por Reviews
list2 = get_top_words(df['Review'], 20, (2,2))

df_words2 = pd.DataFrame(list2, columns=['Word', 'Count'])

plt.figure(figsize=(7,6))
sns.barplot(data=df_words2, x='Count', y='Word')
plt.title('Frequência de pares de palavras para avaliações');

<a id='3'></a>
# <p style="background-color:#2F5597; text-font:white; font-family:newtimeroman; font-size:120%; text-align:center"><span style="color:white">Parte 3: Sistema de Recomendação baseado em conteúdo</span></p>



### TF-IDF Matrix (Term Frequency — Inverse Document Frequency Matrix)

O valor TF-IDF (abreviação do inglês term frequency–inverse document frequency, que significa frequência do termo–inverso da frequência nos documentos), é uma medida estatística que tem o intuito de indicar a importância de uma palavra de um documento em relação a uma coleção de documentos ou em um corpus linguístico. Ela é frequentemente utilizada como fator de ponderação na recuperação de informações e na mineração de dados.

O valor TF-IDF de uma palavra aumenta proporcionalmente à medida que aumenta o número de ocorrências dela no documento, no entanto, esse valor é equilibrado pela frequência da palavra no corpus. Isso auxilia a distinguir o fato da ocorrência de algumas palavras serem geralmente mais comuns que outras.

Assim, o método TF-IDF é usado para quantificar palavras e calcular pesos para elas.

Em outras palavras, representar cada palavra (ou pares de palavras etc.) com um número para usar a matemática no sistema de recomendação.

A similaridade de cosseno é uma métrica usada para determinar quão semelhantes os documentos são, independentemente de seu tamanho.

In [ ]:
# Alterando o índice do conjunto de dados por nome do restaurante
df.set_index('Name', inplace=True)

# Salvando os índices como series
indices = pd.Series(df.index)

# Criando uma matriz tf-idf
# Mais info: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['Review'])

# Calculando a similaridade de cosseno
# O linear_kernel é usado quando os dados são separáveis linearmente, ou seja, podem ser separados usando uma única Linha. 
# É um dos kernels mais comuns a serem usados. É usado principalmente quando há um grande número de recursos em um determinado conjunto de dados
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

## Criando o sistema de recomendação

In [ ]:
def recommend(name, cosine_similarities = cosine_similarities):
    
    # Criar uma lista para por os 10 restaurantes recomendados
    recommend_restaurant = []
    
    # Fixar o índice do hotel inserido
    idx = indices[indices == name].index[0]
    
    # Encontrar os restaurantes com um valor de cosseno semelhante e ordená-los-os do maior para o menor
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending=False)
    
    # Extrair os 30 principais índices de restaurantes com um valor de cosseno semelhante
    top30_indexes = list(score_series.iloc[0:31].index)
    
    # Nomes dos top 30 restaurantes
    for each in top30_indexes:
        recommend_restaurant.append(list(df.index)[each])
    
    # Criar o novo conjunto de dados para mostrar os restaurantes semelhantes
    df_new = pd.DataFrame(columns=['Cuisines', 'Mean Rating', 'Cost', 'Timings'])
    
    # Criar um dataframe com 30 melhores restaurantes semelhantes com algumas de suas colunas
    for each in recommend_restaurant:
        df_new = df_new.append(pd.DataFrame(df[['Cuisines','Mean Rating', 'Cost', 'Timings']][df.index == each].sample()))
    
    # Eliminar os restaurantes com o mesmo nome e classificar apenas os 10 primeiros pela classificação mais alta
    df_new = df_new.drop_duplicates(subset=['Cuisines','Mean Rating', 'Cost'], keep=False)
    df_new = df_new.sort_values(by='Mean Rating', ascending=False).head(10)
    
    print('Top %s restaurantes como %s com avaliações similares: ' % (str(len(df_new)), name))
    
    return df_new

## Testando nosso sistema de recomendação

## 1. Exemplo:

In [ ]:
# AQUI ESTÁ UM RESTAURANTE ALEATÓRIO. VAMOS VER OS DETALHES SOBRE ESSE RESTAURANTE:
df[df.index == 'Hyderabadi Daawat'].head(1)

In [ ]:
# VAMOS VER O QUE O ALGORITMO NOS RECOMENDA
recommend('Hyderabadi Daawat')

## 2. Exemplo:

In [ ]:
# AQUI VAMOS PROVAR UMA PADARIA. VAMOS VER OS DETALHES SOBRE ESSE ESTABELECIMENTO:
df[df.index == 'Labonel'].head(1)

In [ ]:
# VAMOS VER O QUE O ALGORITMO NOS RECOMENDA
recommend('Labonel')

## 3. Exemplo:

In [ ]:
# AQUI ESTÁ UM RESTAURANTE MEDITERRÂNEO / NORTE DA ÍNDIA / KEBAB / CHURRASCO. VAMOS VER OS DETALHES SOBRE ESSE RESTAURANTE:
df[df.index == 'Barbeque Nation'].sample(1)

In [ ]:
# VAMOS VER O QUE O ALGORITMO NOS RECOMENDA
recommend('Barbeque Nation')

### Isso é tudo. Espero que tenham gostado :)